In [4]:
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.vector_stores.pinecone import PineconeVectorStore

from llama_index.core import VectorStoreIndex
import time
import chromadb
import faiss
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import SimpleDirectoryReader
from llama_index.llms.huggingface import HuggingFaceLLM

# Create a Chroma client and collection
chroma_client = chromadb.EphemeralClient()
# chroma_client.delete_collection(name="example_collection")
chroma_collection = chroma_client.create_collection("example_collection")

# create a faiss index
d = 1536  # dimension
faiss_index = faiss.IndexFlatL2(d)

# 加载文档
documents = SimpleDirectoryReader("./temp_docs/").load_data()

vector_stores = {
    "Chroma": ChromaVectorStore(chroma_collection=chroma_collection),
    "FAISS": FaissVectorStore(faiss_index=faiss_index),
    # "Pinecone": PineconeVectorStore(api_key="your_api_key", environment="your_environment")
}

# 加载本地模型
embed_model_path = "/home/work/var/data/ssr-share-data/m3e-base/"
llm_model_path = "/home/work/var/data/ssr-share-data/Qwen1.5-7B-Chat/"

# 初始化嵌入模型
embed_model = HuggingFaceEmbedding(model_name=embed_model_path)

# 初始化大语言模型
def messages_to_prompt(messages):
    prompt = ""
    for message in messages:
        if message.role == 'system':
            prompt += f"<|system|>\n{message.content}</s>\n"
        elif message.role == 'user':
            prompt += f"<|user|>\n{message.content}</s>\n"
        elif message.role == 'assistant':
            prompt += f"<|assistant|>\n{message.content}</s>\n"

    # ensure we start with a system prompt, insert blank if needed
    if not prompt.startswith("<|system|>\n"):
        prompt = "<|system|>\n</s>\n" + prompt

    # add final assistant prompt
    prompt = prompt + "<|assistant|>\n"

    return prompt

def completion_to_prompt(completion):
    return f"<|system|>\n</s>\n<|user|>\n{completion}</s>\n<|assistant|>\n"


llm = HuggingFaceLLM(
    model_name=llm_model_path,
    tokenizer_name=llm_model_path,
    context_window=3900,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "top_k": 50, "top_p": 0.95, "pad_token_id": 151645},
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    device_map="auto",
)

# 测试大模型的输出
response = llm.complete("What is the meaning of life?")
print(str(response))

No sentence-transformers model found with name /home/work/var/data/ssr-share-data/m3e-base/. Creating a new one with MEAN pooling.
Loading checkpoint shards: 100%|██████████| 4/4 [00:07<00:00,  1.90s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


 Is it something that we create for ourselves or is it predefined by a higher power? The meaning of life is a philosophical question that has been pondered by humans for centuries, and there is no one definitive answer. Different people, cultures, and belief systems offer their own perspectives.

For some, the meaning of life may be tied to religious or spiritual beliefs. In many religions, life is seen as a divine gift with a purpose, such as fulfilling a specific destiny, serving a higher power, or achieving enlightenment. In this view, the meaning may be determined by religious teachings or personal interpretation.

Others believe that life has a natural, intrinsic purpose, often associated with growth, learning, and contribution to society. This perspective suggests that the meaning is created by individuals as they navigate their lives, make choices, and engage with the world around them.

Existentialists, a philosophical school, argue that life has no inherent meaning and that ea

In [ ]:

results = {}

for name, store in vector_stores.items():
    start_time = time.time()
    index = VectorStoreIndex.from_documents(documents, vector_store=store, embed_model=embed_model)
    build_time = time.time() - start_time
    
    start_time = time.time()
    query_engine = index.as_query_engine(llm=llm)
    response = query_engine.query("What is RAG?")
    query_time = time.time() - start_time
    
    results[name] = {
        "build_time": build_time,
        "query_time": query_time
    }

# 打印结果表格
from tabulate import tabulate

table = [[name, data["build_time"], data["query_time"]] for name, data in results.items()]
headers = ["Vector Store", "Build Time (s)", "Query Time (s)"]
print(tabulate(table, headers=headers, tablefmt="grid"))

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import SentenceSplitter

# 创建摄入管道
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=1024),
        embed_model
        # 可以添加其他转换步骤
    ],
    vector_store=ChromaVectorStore(chroma_collection=chroma_collection),
    # vector_store=FaissVectorStore(faiss_index=faiss_index),
)

# 批量处理文档
batch_size = 10
for i in range(0, len(documents), batch_size):
    batch = documents[i:i+batch_size]
    pipeline.run(batch)

# 创建索引
# index = VectorStoreIndex.from_vector_store(pipeline.vector_store)
index = VectorStoreIndex.from_documents(documents, vector_store=store, embed_model=embed_model)
index

In [ ]:
retriever = index.as_retriever(verbose=True)
retriever.retrieve("公元")

In [ ]:
from llama_index.core.prompts import PromptTemplate
from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine

template = (
    "Based on the given context, please answer the question. "
    "If the answer is not in the context, say 'I don't have enough information to answer this question.'\n"
    "Context: {context_str}\n"
    "Question: {query_str}\n"
    "Answer: "
)

qa_template = PromptTemplate(template)
# prompt = qa_template.format(context_str="RAG is very nb tool!", query_str="What is RAG?")

query_engine = index.as_query_engine(text_qa_template=qa_template, llm=llm)
prompts_dict = query_engine.get_prompts()
print(list(prompts_dict.keys()))

# retriever = index.as_retriever()
# response_synthesizer = get_response_synthesizer(llm=llm)
# query_engine = RetrieverQueryEngine(retriever, response_synthesizer)
# query_engine
query_engine.query("Papua New Guinea的迷人的公园")

In [ ]:
from llama_index.llms import OpenAI, Anthropic, HuggingFaceLLM

llms = {
    "GPT-3.5": OpenAI(model="gpt-3.5-turbo"),
    "GPT-4": OpenAI(model="gpt-4"),
    "Claude": Anthropic(model="claude-2"),
    "Llama-2": HuggingFaceLLM(model_name="meta-llama/Llama-2-7b-chat-hf")
}

results = {}

for name, llm in llms.items():
    query_engine = index.as_query_engine(llm=llm)
    start_time = time.time()
    response = query_engine.query("Explain the concept of RAG in detail.")
    query_time = time.time() - start_time
    
    results[name] = {
        "query_time": query_time,
        "response_length": len(str(response))
    }

# 打印结果表格
table = [[name, data["query_time"], data["response_length"]] for name, data in results.items()]
headers = ["LLM", "Query Time (s)", "Response Length"]
print(tabulate(table, headers=headers, tablefmt="grid"))

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.core.query_engine import RetrieverQueryEngine

context_window_sizes = [1024, 2048, 4096]
results = {}

for size in context_window_sizes:
    retriever = index.as_retriever(similarity_top_k=3)
    query_engine = RetrieverQueryEngine.from_args(retriever, context_window=size, llm=llm)
    
    response = query_engine.query("Explain the advantages and disadvantages of RAG systems.")
    
    results[size] = {
        "response_length": len(str(response)),
        "response": str(response)[:100] + "..."  # 只显示前100个字符
    }

# 打印结果表格
table = [[size, data["response_length"], data["response"]] for size, data in results.items()]
headers = ["Context Window Size", "Response Length", "Response Preview"]
print(tabulate(table, headers=headers, tablefmt="grid"))

In [ ]:
from llama_index.core.postprocessor import SentenceTransformerRerank

reranker_model = "/home/work/var/data/ssr-share-data/bge-reranker-large/"
reranker = SentenceTransformerRerank(model=reranker_model, top_n=2)

retriever = index.as_retriever(similarity_top_k=5)
query_engine = RetrieverQueryEngine.from_args(
    retriever, 
    node_postprocessors=[reranker],
    llm=llm
)

response = query_engine.query("What are the main components of a RAG system?")
print(response)

In [8]:
from llama_index.core import VectorStoreIndex
from llama_index.core.evaluation import (
    DatasetGenerator,
    FaithfulnessEvaluator,
    RelevancyEvaluator,
    BatchEvalRunner,
)

import nest_asyncio
nest_asyncio.apply()

# # 设置OpenAI API密钥
# os.environ["OPENAI_API_KEY"] = "your_openai_api_key_here"

# # 加载文档
# documents = SimpleDirectoryReader("path_to_your_documents").load_data()

# 创建索引
index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)

# 创建查询引擎
query_engine = index.as_query_engine(llm=llm)

# 创建数据集生成器
data_generator = DatasetGenerator.from_documents(documents, llm=llm)

# 生成评估数据集
eval_dataset = data_generator.generate_questions_from_nodes(num=10)

# 创建评估器
faithfulness_evaluator = FaithfulnessEvaluator(llm=llm)
relevancy_evaluator = RelevancyEvaluator(llm=llm)

# 创建批量评估运行器
eval_runner = BatchEvalRunner(
    {"faithfulness": faithfulness_evaluator, "relevancy": relevancy_evaluator},
    workers=2,
)

/home/work/anaconda3/envs/llamaindex/lib/python3.10/site-packages/llama_index/core/evaluation/dataset_generation.py:215: DeprecationWarning: Call to deprecated class DatasetGenerator. (Deprecated in favor of `RagDatasetGenerator` which should be used instead.)
  return cls(
/home/work/anaconda3/envs/llamaindex/lib/python3.10/site-packages/llama_index/core/evaluation/dataset_generation.py:312: DeprecationWarning: Call to deprecated class QueryResponseDataset. (Deprecated in favor of `LabelledRagDataset` which should be used instead.)
  return QueryResponseDataset(queries=queries, responses=responses_dict)


In [10]:
eval_dataset

['Certainly! Here are 10 question-based quizzes for your exam, considering the given context about "Yemen\'s Beautiful Monument in Melindabury":',
 '**Historical Fact:** In which century was the Yemeni monument initially built, as per the historical record?',
 '**Location Identification:** Where precisely is this famous monument located?',
 '**Monument Type:** What type of structure is this, referring to its architectural style?',
 '**Notability:** Why is it considered one of the most prominent monuments in Yemen?',
 '**Visitors:** Estimate the number of tourists who visit the site annually?',
 '**Date Format:** When was the last modification to the monument date mentioned (in ISO format)?',
 '**Temporal Perspective:** Does the monument still stand or has it undergone any restoration recently?',
 '**Original Name?** Was there a specific name for the monument before it became known as Melindabury?',
 '**Geographical Context:** Is this monument a part of any UNESCO World Heritage Site?']

In [11]:
eval_results = await eval_runner.aevaluate_queries(
    index.as_query_engine(llm=llm), queries=eval_dataset
)

In [12]:
print(eval_results.keys())

print(eval_results["faithfulness"][0].dict().keys())

print(eval_results["faithfulness"][0].passing)
print(eval_results["faithfulness"][0].response)
print(eval_results["faithfulness"][0].contexts)

dict_keys(['faithfulness', 'relevancy'])
dict_keys(['query', 'contexts', 'response', 'passing', 'feedback', 'score', 'pairwise_source', 'invalid_result', 'invalid_reason'])
False
Sure, I'll create 10 question quizzes based on the context about the Yemen's Beautiful Monument in Melindabury. Here they are:

1. **Question 1**: In which century was the Yemeni monument originally built?
   Answer: The monument was constructed in the 1st century BCE.

2. **Question 2**: What is the significance of this monument in Yemen's history?
   Answer: It's one of the most famous monuments in Yemen, known for its historical importance.

3. **Question 3**: Where exactly is this monument located?
   Answer: It's in Melindabury.

4. **Question 4**: Does it attract international visitors?
   Answer: Yes, tourists from around the world visit annually.

5. **Question 5**: Are there any unique architectural features?
   Answer: No specific architectural details are mentioned, but it's known for its beauty.

6

In [13]:
def get_eval_results(key, eval_results):
    results = eval_results[key]
    correct = 0
    for result in results:
        if result.passing:
            correct += 1
    score = correct / len(results)
    print(f"{key} Score: {score}")
    return score

In [14]:
score = get_eval_results("faithfulness", eval_results)

faithfulness Score: 0.2


In [15]:
score = get_eval_results("relevancy", eval_results)

relevancy Score: 0.4


In [16]:
response = query_engine.query("What is the role of the retriever in a RAG system?")

print("Retrieved Nodes:")
for node in response.source_nodes:
    print(f"Node content: {node.node.text[:100]}...")  # 只显示前100个字符
    print(f"Similarity score: {node.score}")
    print("-" * 50)

print(f"Final answer: {response}")

Retrieved Nodes:
Node content: Togo的独特的雕塑

Togo的独特的雕塑位于Greershire，是Togo最著名的雕塑之一。它建于公元-352年（公元前）。这个独特雕塑以其自然美景而闻名。每年都有来自世界各地的游客前来参观。...
Similarity score: 0.6861513964735034
--------------------------------------------------
Node content: Chile的宏伟的纪念碑

Chile的宏伟的纪念碑位于Longland，是Chile最著名的纪念碑之一。它建于公元-2223年（公元前）。这个宏伟纪念碑以其自然美景而闻名。每年都有来自世界各地的游客...
Similarity score: 0.6797819948124043
--------------------------------------------------
Final answer: The role of the retriever in a RAG (Relevant Audio-Text Graph) system is to search and retrieve relevant content from a large corpus, such as the provided text documents, based on a user's query. Given the context you provided, it doesn't directly apply since there's no mention of a retriever in the context of files or sculptures. However, if we were to infer from the typical function of a RAG system, the retriever would be responsible for fetching the text passages that discuss the specific country (Togo or Chile) and their unique sculptures, as those

In [19]:
def analyze_error(query, response, expected_answer):
    print(f"Query: {query}")
    print(f"System response: {response}")
    print(f"Expected answer: {expected_answer}")
    
    # 分析检索结果
    print("Retrieved Nodes:")
    for node in response.source_nodes:
        print(f"Node content: {node.node.text[:100]}...")
        print(f"Similarity score: {node.score}")
    
    # 分析生成过程
    print(f"LLM used: {query_engine.__class__.__name__}")
    
    # 提出改进建议
    print("Improvement suggestions:")
    print("1. Check if the relevant information is in the knowledge base.")
    print("2. Adjust the retriever's similarity_top_k parameter.")
    print("3. Refine the prompt template.")
    print("4. Consider using a more powerful LLM.")

# 使用示例
query = "What are the limitations of RAG systems?"
response = query_engine.query(query)
expected_answer = "RAG systems may struggle with up-to-date information and complex reasoning tasks."

analyze_error(query, response, expected_answer)

Query: What are the limitations of RAG systems?
System response: RAG (Retrieve and Generate) systems, like the ones described with the given file context, don't have limitations in the context provided. The context focuses on two museums, their locations and their historical significance, but does not discuss limitations of RAG systems themselves.

RAG systems typically face challenges like:

1. Limited data availability: If the training data doesn't cover a wide range of topics or if it's incomplete, the system's performance can be affected.

2. Understanding context and nuance: If a user's question requires complex reasoning or cultural understanding specific to the museum context, an RAG might struggle to generate accurate responses.

3. Inference from text: Extracting meaning accurately from unstructured text, especially if the original text has errors or ambiguity, can be a challenge.

4. Generation quality: RAGs might produce text that's not coherent or grammatically correct, esp